# Versión tres del modelo (Regresión logistica)
Voy a probar ahora el otro modelo, pero con los mismos datos que en la v2.

Hay que solucionar todavía lo del pd.merge()

Update1 : Va mejor que el RandomForest de la versión v2 (para las mismas variables). Precisión del modelo: 0.4558315391790497

Update2: Le he metido una variable más que es el equipo home. No cambia casi nada. Puede que sea porque no le he metido muchos datos para el train. Precisión del modelo: 0.45270325763664415

In [75]:
# Importo los paquetes
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression # el modelo
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # para ver cómo funciona
from sklearn.preprocessing import LabelEncoder # poner los datos tipo int



In [96]:
# Importo los dataframes
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close() 

In [83]:
# Importo los dataframes
with sqlite3.connect("clasification.sqlite") as conn:
        df_teams = pd.read_sql("SELECT * FROM clasification", conn)
conn.close()  

In [97]:
# Si pongo directamente df_games =con(...) me da error en la siguiente celda
df_games=df  

In [98]:
# Le creo a df_games la columna de result (home, visitor, tie) que va a ser mi variable Y en el modelo
df_games["result"] = None

df_games.loc[(df_games["score"].str.split(":").str[0]) > (df_games["score"].str.split(":").str[1]), "result"] = 'home'
df_games.loc[(df_games["score"].str.split(":").str[0]) == (df_games["score"].str.split(":").str[1]), "result"] = 'tie'
df_games.loc[(df_games["score"].str.split(":").str[0]) < (df_games["score"].str.split(":").str[0]), "result"] = 'visitor'

In [99]:
df_games.drop(["date", "time", "score"], axis=1, inplace=True)
df_games

,season,division,matchday,home_team,away_team,result
0,1928-1929,1,1,Arenas Club,Athletic Madrid,None
1,1928-1929,1,1,Espanyol,Real Unión,home
2,1928-1929,1,1,Real Madrid,Catalunya,home
3,1928-1929,1,1,Donostia,Athletic,tie
4,1928-1929,1,1,Racing,Barcelona,None
...,...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,CD Tenerife,FC Cartagena,None


In [139]:
df_games_home = df_games[["season", "division", "matchday", "home_team", "result"]]
df_games_visitor = df_games[["season", "division", "matchday", "away_team", "result"]]
df_games_home

,season,division,matchday,home_team,result
0,1928-1929,1,1,Arenas Club,None
1,1928-1929,1,1,Espanyol,home
2,1928-1929,1,1,Real Madrid,home
3,1928-1929,1,1,Donostia,tie
4,1928-1929,1,1,Racing,None
...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,None
48776,2021-2022,2,42,Real Sociedad B,None
48777,2021-2022,2,42,Sporting Gijón,None
48778,2021-2022,2,42,CD Tenerife,None


# Empieza el modelo (Logistic regression)
Nuestro modelo/problema es de tipo clasificación. La predicción nos tiene que dar uno de los tres valores, NO es una variable continua.

Ya tenemos los datos. Ahora hay que definir nuestras variables. X son los datos que vamos a tener en cuenta, e Y es la "variable etiqueta", es decir, la que tenemos que predecir. La etiqueta tiene que ser de tipo int, empezamos con eso.

In [46]:
division = 1
season = "2000-2001"
matchday = 3
season_train = "2019:2020"


In [47]:
# Luego, aplicar las operaciones de cadena de caracteres
df_teams_train = df_teams

# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres
df_teams_train["season"] = df_teams_train["season"].astype(str)

df_teams_train = df_teams_train[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_teams_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_teams_train


,season,division,rank,team,matchday,GF,GA,GD,W,L,T,Pts
3368,2019-2020,1,1.0,Real Madrid,1,3.0,1.0,2.0,1.0,0.0,0.0,3.0
3369,2019-2020,1,2.0,Sevilla FC,1,2.0,0.0,2.0,1.0,0.0,0.0,3.0
3370,2019-2020,1,3.0,RCD Mallorca,1,2.0,1.0,1.0,1.0,0.0,0.0,3.0
3371,2019-2020,1,4.0,Real Valladolid,1,2.0,1.0,1.0,1.0,0.0,0.0,3.0
3372,2019-2020,1,5.0,Alavés,1,1.0,0.0,1.0,1.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5047,2019-2020,2,18.0,Ponferradina,42,45.0,50.0,-5.0,12.0,15.0,15.0,51.0
5048,2019-2020,2,19.0,Dep. La Coruña,42,43.0,60.0,-17.0,12.0,15.0,15.0,51.0
5049,2019-2020,2,20.0,CD Numancia,42,45.0,53.0,-8.0,13.0,18.0,11.0,50.0
5050,2019-2020,2,21.0,Extremadura,42,43.0,59.0,-16.0,10.0,19.0,13.0,43.0


In [140]:
df_games_home_train = df_games_home
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_home_train["season"] = df_games_home_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_home_train = df_games_home_train[(df_games_home_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_home_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_games_home_train


/tmp/ipykernel_5916/4240325088.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_home_train["season"] = df_games_home_train["season"].astype(str)


,season,division,matchday,home_team,result
25058,2019-2020,1,1,Athletic,home
25059,2019-2020,1,1,Celta de Vigo,None
25060,2019-2020,1,1,Valencia,tie
25061,2019-2020,1,1,RCD Mallorca,home
25062,2019-2020,1,1,CD Leganés,None
...,...,...,...,...,...
47471,2019-2020,2,42,CD Numancia,home
47472,2019-2020,2,42,Racing,None
47473,2019-2020,2,42,Sporting Gijón,None
47474,2019-2020,2,42,Real Zaragoza,home


In [142]:
df_games_visitor_train = df_games_visitor
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_visitor_train["season"] = df_games_visitor_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_visitor_train = df_games_visitor_train[(df_games_visitor_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_visitor_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_games_visitor_train

/tmp/ipykernel_5916/3908699393.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_visitor_train["season"] = df_games_visitor_train["season"].astype(str)


,season,division,matchday,away_team,result
25058,2019-2020,1,1,Barcelona,home
25059,2019-2020,1,1,Real Madrid,None
25060,2019-2020,1,1,Real Sociedad,tie
25061,2019-2020,1,1,SD Eibar,home
25062,2019-2020,1,1,CA Osasuna,None
...,...,...,...,...,...
47471,2019-2020,2,42,CD Tenerife,home
47472,2019-2020,2,42,Rayo Vallecano,None
47473,2019-2020,2,42,SD Huesca,None
47474,2019-2020,2,42,Ponferradina,home


In [148]:
df_train_home = pd.merge(df_games_home_train, df_teams_train, how="outer", 
                    left_on=['home_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    suffixes=("_home", ""))

df_train_home.dropna(subset="home_team", inplace=True)

In [150]:
df_train_away = pd.merge(df_games_visitor_train, df_teams_train, 
                    left_on=['away_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    how='outer', suffixes=("_away",""))
df_train_away.dropna(subset="away_team", inplace=True)

In [154]:
df_train = pd.concat([df_train_home, df_train_away])
df_train

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team
0,2019-2020,1,1,Athletic,home,6.0,Athletic,1.0,0.0,1.0,1.0,0.0,0.0,3.0,NaN
1,2019-2020,1,1,Celta de Vigo,None,19.0,Celta de Vigo,1.0,3.0,-2.0,0.0,1.0,0.0,0.0,NaN
2,2019-2020,1,1,Valencia,tie,12.0,Valencia,1.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN
3,2019-2020,1,1,RCD Mallorca,home,3.0,RCD Mallorca,2.0,1.0,1.0,1.0,0.0,0.0,3.0,NaN
4,2019-2020,1,1,CD Leganés,None,16.0,CD Leganés,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,2019-2020,2,42,NaN,home,12.0,CD Tenerife,50.0,46.0,4.0,14.0,15.0,13.0,55.0,CD Tenerife
838,2019-2020,2,42,NaN,None,7.0,Rayo Vallecano,60.0,50.0,10.0,13.0,8.0,21.0,60.0,Rayo Vallecano
839,2019-2020,2,42,NaN,None,1.0,SD Huesca,55.0,42.0,13.0,21.0,14.0,7.0,70.0,SD Huesca
840,2019-2020,2,42,NaN,home,18.0,Ponferradina,45.0,50.0,-5.0,12.0,15.0,15.0,51.0,Ponferradina


In [136]:
# Hacemos el merge del dataframe para train
df_train = pd.merge(df_games_train, df_teams_train, left_on=['home_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], how='outer', suffixes=("_home", ""))
df_train = pd.merge(df_train, df_teams_train, left_on='away_team', right_on='team', how='left', suffixes=("_away",""))
df_train

,season,division,matchday,home_team,away_team,result,rank,team,GF,GA,GD,W,L,T,Pts
0,2019-2020,1,1,Athletic,Barcelona,home,6.0,Athletic,1.0,0.0,1.0,1.0,0.0,0.0,3.0
1,2019-2020,1,1,Celta de Vigo,Real Madrid,None,19.0,Celta de Vigo,1.0,3.0,-2.0,0.0,1.0,0.0,0.0
2,2019-2020,1,1,Valencia,Real Sociedad,tie,12.0,Valencia,1.0,1.0,0.0,0.0,0.0,1.0,1.0
3,2019-2020,1,1,RCD Mallorca,SD Eibar,home,3.0,RCD Mallorca,2.0,1.0,1.0,1.0,0.0,0.0,3.0
4,2019-2020,1,1,CD Leganés,CA Osasuna,None,16.0,CD Leganés,0.0,1.0,-1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,2019-2020,2,42,NaN,NaN,NaN,14.0,Málaga CF,35.0,33.0,2.0,11.0,11.0,20.0,53.0
1680,2019-2020,2,42,NaN,NaN,NaN,15.0,Real Oviedo,49.0,53.0,-4.0,13.0,15.0,14.0,53.0
1681,2019-2020,2,42,NaN,NaN,NaN,16.0,Albacete,36.0,46.0,-10.0,13.0,16.0,13.0,52.0
1682,2019-2020,2,42,NaN,NaN,NaN,18.0,Ponferradina,45.0,50.0,-5.0,12.0,15.0,15.0,51.0


In [50]:
# Supongamos que tienes un DataFrame df con una columna 'etiqueta' de tipo 'object'
le = LabelEncoder()
df_train['prediction'] = le.fit_transform(df_train['result'])
df_train["home"] = le.fit_transform(df_train["home_team"])

In [51]:
# Selecciona las características (X_train) y etiquetas (y_train)
X_train = df_train[['home', 'rank', 'GD', "W", "Pts"]]  
y_train = df_train['prediction']  

In [63]:
df_teams_test = df_teams[(df_teams["season"] == season) & (df_teams["division"] == division) & (df_teams["matchday"] == matchday)]
df_games_test = df_games[(df_games["season"] == season) & (df_teams["division"] == division) & (df_teams["matchday"] == matchday)]

/tmp/ipykernel_5916/3707183071.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_games_test = df_games[(df_games["season"] == season) & (df_teams["division"] == division) & (df_teams["matchday"] == matchday)]


In [64]:
# Hacemos el merge del dataframe para hacer el test
df_test = pd.merge(df_games_test, df_teams_test, left_on='home_team', right_on='team', how='left', suffixes=("_home", ""))
df_test = pd.merge(df_test, df_teams_test, left_on='away_team', right_on='team', how='left', suffixes=("_away",""))
df_test

,season_home,division_home,matchday_home,home_team,away_team,result,season_away,division_away,rank_away,team_away,...,rank,team,matchday,GF,GA,GD,W,L,T,Pts


In [62]:
le = LabelEncoder()
df_test['prediction'] = le.fit_transform(df_test['result'])
df_test["home"] = le.fit_transform(df_test["home_team"])
df_test

,season_home,division_home,matchday_home,home_team,away_team,result,season_away,division_away,rank_away,team_away,...,matchday,GF,GA,GD,W,L,T,Pts,prediction,home
0,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,1,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1,28
1,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,2,2.0,5.0,-3.0,0.0,1.0,1.0,1.0,1,28
2,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,3,3.0,8.0,-5.0,0.0,2.0,1.0,1.0,1,28
3,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,4,3.0,8.0,-5.0,0.0,2.0,2.0,2.0,1,28
4,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,5,4.0,10.0,-6.0,0.0,3.0,2.0,2.0,1,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363683,2000-2001,2,42,Getafe,Atlético Madrid,visitor,2000-2001,2,21.0,Getafe,...,38,55.0,39.0,16.0,17.0,10.0,11.0,62.0,2,16
1363684,2000-2001,2,42,Getafe,Atlético Madrid,visitor,2000-2001,2,21.0,Getafe,...,39,56.0,39.0,17.0,18.0,10.0,11.0,65.0,2,16
1363685,2000-2001,2,42,Getafe,Atlético Madrid,visitor,2000-2001,2,21.0,Getafe,...,40,57.0,39.0,18.0,19.0,10.0,11.0,68.0,2,16
1363686,2000-2001,2,42,Getafe,Atlético Madrid,visitor,2000-2001,2,21.0,Getafe,...,41,58.0,39.0,19.0,20.0,10.0,11.0,71.0,2,16


In [55]:
X_test = df_test[["home",'rank', 'GD', "W", "Pts"]] # he cogido estás aleatoriamente
y_test = df_test['prediction']

In [56]:
# Crear y entrenar un modelo de clasificación (por ejemplo, Random Forest)
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [57]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [61]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, target_names=['home_win(1)', 'visitor_win(2)', 'tie(X)'])

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion_mat}")
print(f"Classification Report:\n{class_report}")

Accuracy: 0.4680469432890808
Confusion Matrix:
[[486112  82838  58858]
 [273144  87306  54162]
 [216928  39488  64852]]
Classification Report:
                precision    recall  f1-score   support

   home_win(1)       0.50      0.77      0.61    627808
visitor_win(2)       0.42      0.21      0.28    414612
        tie(X)       0.36      0.20      0.26    321268

      accuracy                           0.47   1363688
     macro avg       0.43      0.40      0.38   1363688
  weighted avg       0.44      0.47      0.43   1363688

